In [1]:
import pandas as pd
import numpy as np
import wikipedia
import json
import simplejson
import urllib as urllib
import urllib.request, urllib.parse, urllib.error
import mwparserfromhell
import pywikibot
import re

In [2]:
missions = pd.read_csv('mission_data_raw.csv', delimiter=',', encoding='utf-8')
mission_count = len(missions['Vehicle'])

Cleaning the mission_orbital_data first
This is primarity a problem of string formatting and can be improved by reading the data in it's original language and parsing appropriately

In [3]:
#Cleaning 'Vehicle'
vehicles = missions['Vehicle']
vehicles_formatted = []

for i in range(mission_count):
    
    #strips extra spaces
    a = vehicles[i].strip()
    vehicles_formatted.append(a)
    

In [4]:
#Cleaning  'mission_duration'
mission_duration = missions['mission_duration']
mission_duration_formatted = []

for i in range(mission_count):
    a = str(mission_duration[i])

    #Replaces a substring '&nbsp;' read inconsistently from ASCII
    a = a.replace("&nbsp;", " ") 
    
    #Replaces string on text from ISS missions
    a = a.replace("(launch to landing)</small>", "") 
    
    #Replaces string on text from ISS missions
    a = a.replace("(at ISS)<br/>", ", ") 
    
    #Starts Converting text from some javascript we don't want
    a = a.replace("{{", "<") 
    
    #Finishes converting text from javascript we don't want
    a = a.replace("}}", ">,") 
    
    #Deletes anything within < >
    a = re.sub('<[^>]+>', '', a) 
    
    #Deletes anything within parenthesis
    a = re.sub("[\(\[].*?[\)\]]", "", a) 
    
    #Replaces newline characters
    a = a.replace("\n", " ") 
    a = a.replace(".", "")
    
    #formats the text for the time units to be uniform for the dictionary
    a = a.replace("months", "month") 
    a = a.replace("hours", "hour")
    a = a.replace("days", "day")
    a = a.replace("minutes", "minute")
    a = a.replace("seconds", "second")
    
    #Splits the cleaned string into a readable format
    a = a.strip().split(",")
    mission_duration_formatted.append(a)    

In [5]:
#Defines a function to convert a dict of times from 'mission_duration' to a single float for ease of calculation
def toseconds(dict):     
        
        #converts months to days with the AVERAGE number of days in a month
        #Not a good solution and may be off by days. Fix requires a calendar lookup for each mission
        d1 = 30.4375 * dict['month']
        d = 86400 * (d1+ dict['day'])
        h = 3600 * dict['hour']
        m = 60 * dict['minute']
        s = dict['second']
        total = (d + h + m + s)
        return(total)

In [6]:
#Initializes a list for a converted duration measurement
list_of_time_dicts = []


for i in range(mission_count): #Iterate through each mission
    
    #Initialize a dictionary of each time measurement
    duration_dict = {'month':0, 'day':0, 'hour':0, 'minute':0, 'second':0} 
    
    #Iterate through each "n timeunit" element
    for j in range(len(mission_duration_formatted[i])): 
        
        #Split the number from the unit
        parcel = mission_duration_formatted[i][j].strip().split() 
        
        #Check if the string is wrongly formatted
        if len(parcel) < 2: 
            
            #Append empty data so the iteration can complete
            parcel = ['0', 'NaN'] 
        else:
            
            #Set the dictionary category to the input digits
            duration_dict[parcel[1]] = int(parcel[0]) 
        
    list_of_time_dicts.append(duration_dict)
        
        
    #Check for any dicts that received a wrongly formatted dictionary
    #Dicts that did not pass with correct dicts will print below
    if len(duration_dict) != 5:
        print(i, duration_dict)

77 {'month': 0, 'day': 15, 'hour': 0, 'minute': 0, 'second': 11, 'min': 47}


In [7]:
total_duration_in_seconds = []
duration_months = []
duration_days = []
duration_hours = []
duration_minutes = []
duration_seconds = []
for i in range(len(list_of_time_dicts)):
    secs = toseconds(list_of_time_dicts[i])
    total_duration_in_seconds.append(secs)
    duration_months.append(list_of_time_dicts[i]['month'])
    duration_days.append(list_of_time_dicts[i]['day'])
    duration_hours.append(list_of_time_dicts[i]['hour'])
    duration_minutes.append(list_of_time_dicts[i]['minute'])
    duration_seconds.append(list_of_time_dicts[i]['second'])

#total_duration_in_seconds
#duration_seconds

Cleaning the 'orbits_completed' category

In [8]:
orbits_completed = missions['orbits_completed']
orbits_completed
orbits_completed_cleaned = []

for i in range(mission_count):
    orbits = str(orbits_completed[i])
    #orbits = re.sub("[\(\[].*?[\)\]]", "", orbits)
    orbits = orbits.replace(',', '')
    orbits = orbits.replace('<ref', ' ')
    orbits = orbits.replace('~', '')
    orbits = orbits.replace('nan', '0')
    orbits = orbits.strip().split()
    if len(orbits) == 0:
        orbits = 0
    else:
        orbits = int(orbits[0])
    
    orbits_completed_cleaned.append(orbits)

In [9]:
#Cleaning 'orbit_periapsis'

orbit_periapsis = missions['orbit_periapsis']
length_dict = {}
periapsis_in_miles = []

for i in range(mission_count):
    periapsis = str(orbit_periapsis[i])
    periapsis = periapsis.replace('}}\r\n', '')
    
    periapsis = periapsis.split('|')
    
    
    #This loop searches for a string that occurs immediately before the unit of measurement by default on the infobox
    #This caves a dict with both a value and the unit of measurement.
    if periapsis[0] == ' {{convert':
        
        if '}}<' in periapsis[2]:
            periapsis[2].split('}}<')
            length_dict['type'] = periapsis[2][0]
        else:
            length_dict['type'] = periapsis[2]
        length_dict['dist'] = float(periapsis[1])
    elif periapsis[0] == 'nan':
        length_dict['type'] = 'mi'
        length_dict['dist'] = 0
    
    #Converts kilomaters to standard miles
    if length_dict['type'] == 'km' or length_dict['type'] == 'k':
        length_dict['dist'] = length_dict['dist'] * 0.62137119
        length_dict['type'] = 'mi'
    
    #Converts nautical miles to standard miles
    if length_dict['type'] == 'nmi':
        length_dict['dist'] = length_dict['dist'] * 1.15077945
        length_dict['type'] = 'mi'
    
    
    periapsis_in_miles.append(length_dict['dist']) 

In [10]:
#Cleaning 'orbit_apoapsis'

#This entire cell is an exact copy of that used for periapsis, with names swapped. 
orbit_apoapsis = missions['orbit_apoapsis']
#orbit_apoapsis

length_dict = {}
apoapsis_in_miles = []

for i in range(mission_count):
    apoapsis = str(orbit_apoapsis[i])
    apoapsis = apoapsis.replace('}}\r\n', '')

    
    apoapsis = apoapsis.split('|')
    
    if apoapsis[0] == ' {{convert':
        
        if '}}<' in apoapsis[2]:
            apoapsis[2].split('}}<')
            length_dict['type'] = apoapsis[2][0]
        else:
            length_dict['type'] = apoapsis[2]
        length_dict['dist'] = float(apoapsis[1])
    elif apoapsis[0] == 'nan':
        length_dict['type'] = 'mi'
        length_dict['dist'] = 0
    
    if length_dict['type'] == 'km' or length_dict['type'] == 'k':
        length_dict['dist'] = length_dict['dist'] * 0.62137119
        length_dict['type'] = 'mi'
    
    if length_dict['type'] == 'nmi':
        length_dict['dist'] = length_dict['dist'] * 1.15077945
        length_dict['type'] = 'mi'
    
    apoapsis_in_miles.append(length_dict['dist'])    

In [11]:
#Cleaning 'orbit_inclination'

orbit_inclination = missions['orbit_inclination']
orbit_inclination_cleaned = []

for i in range(mission_count):
    inclination = str(orbit_inclination[i])
    inclination = inclination.replace('&nbsp;', ' ')
    inclination = inclination.replace('nan', '0')
    inclination = inclination.replace('°', ' ')
    inclination = inclination.split()
    
    
    if len(inclination) == 0:
        inclination = '0'
    
    orbit_inclination_cleaned.append(eval(inclination[0]))

In [12]:
#Cleaning 'orbital_period'

orbit_period = missions['orbit_period']
orbit_period_cleaned = []
for i in range(mission_count):
    period = str(orbit_period[i])
    period = period.replace('&nbsp;', ' ')
    period = period.replace('nan', '0')
    period = period.replace('°', ' ')
    period = period.split()
    if len(period) == 0:
        period = '0'
    
    orbit_period_cleaned.append(eval(period[0]))

This is not a data cleaning step, but a data appending step to fill in ISS data which was not included on each Expedition's data. Orbital parameters for ISS taken from https://en.wikipedia.org/wiki/International_Space_Station to maintain consistence with other mission data. 

https://arstechnica.com/science/2013/07/how-nasa-steers-the-international-space-station-around-space-junk/
ISS Experiences drag and is navigated through orbit, so it's paramaters are changing. 

In [13]:
ISSPeriapsis = 249.2
ISSApoapsis = 253.5
ISSInclination = 51.64
ISSPeriod = 92.65

In [14]:
for i in range(mission_count):
    vehicle = str(missions['Vehicle'][i])
    if "Expedition" in vehicle:
        #print(vehicle)
        periapsis_in_miles[i] = ISSPeriapsis
        apoapsis_in_miles[i] = ISSApoapsis
        orbit_inclination_cleaned[i] = ISSInclination
        orbit_period_cleaned[i] = ISSPeriod

In [15]:
#Creates a new DataFRame with cleaned orbital parameter data
df = pd.DataFrame(missions['Vehicle'])

df['Vehicle'] = pd.Series(vehicles_formatted)
df['orbit_periapsis'] = pd.Series(periapsis_in_miles)
df['orbit_apoapsis'] = pd.Series(apoapsis_in_miles)
df['orbit_inclination'] = pd.Series(orbit_inclination_cleaned)
df['orbit_period'] = pd.Series(orbit_period_cleaned)
df['orbit_completed'] = pd.Series(orbits_completed_cleaned)
df['duration_months'] = pd.Series(duration_months)
df['duration_days'] = pd.Series(duration_days)
df['duration_hours'] = pd.Series(duration_hours)
df['duration_minutes'] = pd.Series(duration_minutes)
df['duration_seconds'] = pd.Series(duration_seconds)
df['total_duration_in_seconds'] = pd.Series(total_duration_in_seconds)

orbitalmissiondata = df
df.head(5)

,Vehicle,orbit_periapsis,orbit_apoapsis,orbit_inclination,orbit_period,orbit_completed,duration_months,duration_days,duration_hours,duration_minutes,duration_seconds,total_duration_in_seconds
0,Gemini IV,102.526246,179.576274,32.5,89.03,66,0,4,1,56,12,352572.0
1,Gemini VIII,162.177881,167.770221,28.9,89.81,6,0,0,10,41,26,38486.0
2,Gemini IX-A,169.012964,170.255706,28.8,89.97,47,0,3,0,20,50,260450.0
3,Gemini X,185.789986,469.756620,28.8,95.19,43,0,2,22,46,39,254799.0
4,Gemini XI,185.168615,850.035788,28.8,101.57,44,0,2,23,17,9,256629.0


Cleaning of EVA Dataset begins here

In [16]:
spacewalks = pd.read_csv('Extra-vehicular_Activity__EVA__-_US_and_Russia.csv', delimiter=',', encoding='utf-8')

In [17]:
spacewalks.head(5)
vehicles = spacewalks['Vehicle']
count = len(vehicles)
count

377

In [18]:
vehicles_cleaned = []
for i in range(count):
    a = str(vehicles[i])
    a = a.replace("ISS-Incr ", "Expedition ")
    a = a.replace("ISS Incr-", "Expedition ")           
    a = a.replace("Incr-", "Expedition ")
    a = a.strip().split("   ")
    a = a[0].strip().split("/")
    a = a[0].strip().split("during")
    a = a[0]
    vehicles_cleaned.append(a)
    #print(a)

In [19]:
crew_jumbled = spacewalks['Crew']
crew_0 = []
crew_1 = []
crew_2 = []

for i in range(count):
    crew = crew_jumbled[i].strip().split("  ")
    crew = list(filter(bool, crew))
    n = len(crew)
    for j in range(n):
        crew[j] = crew[j].strip()
    
    if n == 1:
        crew_0.append(crew[0])
        crew_1.append("NaN")
        crew_2.append("NaN")
    
    elif n == 2:
        crew_0.append(crew[0])
        crew_1.append(crew[1])
        crew_2.append("NaN")
    
    elif n == 3:
        crew_0.append(crew[0])
        crew_1.append(crew[1])
        crew_2.append(crew[2])        
        
    else:
        print("Too many crewman")
        


In [20]:
import time

def month_converter(month):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
    if month in months:
        return months.index(month) + 1
    elif month not in months:
        return(month)
    
def month_converter_full(month):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    if month in months:
        return months.index(month) + 1
    elif month not in months:
        return(month)
    
    
EVA_start_year = []
EVA_start_month = []
EVA_start_day = []


for i in range(count):
    d = spacewalks['Date'][i]
    d = d.replace('/', ' ')
    d = d.replace(',', '')
    d = d.replace('.', '')
    d = d.strip().split()
    
    if "-" in d[1]:
        d[1] = d[1].strip().split("-")
        d[1] = str(d[1][0])
   
    if d[0].isdigit() == True:
        d[0] = eval(d[0])
    
    else:
        d[0] = str(month_converter(d[0]))
        
        if d[0].isdigit == True:
            d[0] = eval(d[0])
        else:
            d[0] = month_converter_full(d[0])
            d[0] = int(d[0])
    
    

    #Error checking for any wrongly formatted dates.
    if len(d) != 3:
        print(i, d)
    d[1] = eval(d[1])
    d[2] = eval(d[2])
    
    EVA_start_year.append(d[2])
    EVA_start_month.append(d[0])
    EVA_start_day.append(d[1])

    
    #Errors were found at indexes 238 and 258 which needed appending within the original dataset.
    #Line 238 was a Soyuz mission where the NASA data failed to include an EVA day
    #Line 258 was Soyux mission where a typo made data and year pushed together
    #[238]
    #[258]

In [21]:
was_moonshot = []
#This relies on the specific strings for the Apollo missions which went to the moon.
moonshots = ['Apollo 10', 'Apollo 11', 'Apollo 12', 'Apollo 13', 'Apollo 14', 'Apollo 15', 'Apollo 16', 'Apollo 17']
for i in range(count):
    if spacewalks['Vehicle'][i] in moonshots:
        was_moonshot.append(True)
    else:
        was_moonshot.append(False)   

In [22]:
#Calculates spacewalk duration in seconds
spacewalk_duration = spacewalks['Duration']
spacewalk_durationS_in_sec = []
for i in range(count):
    a = spacewalk_duration[i].replace(':', ' ')
    a = a.strip().split()
    spacewalk_duration_in_seconds = (int(a[0])*3600) + (int(a[1])*60)
    spacewalk_durationS_in_sec.append(spacewalk_duration_in_seconds)
    

In [23]:
#Creates a new mission_type category out of the Vehicle name.
vehicle = spacewalks['Vehicle']
mission_type = []
for i in range(count):
    a = str(vehicle[i])
    a = a.replace("-", " ")
    a = a.split(' ')
    print(a)
    mission_type.append(a[0])

['Gemini', 'IV']
['Gemini', 'VIII']
['Gemini', 'IX', 'A']
['Gemini', 'X']
['Gemini', 'X']
['Gemini', 'XI']
['Gemini', 'XI']
['Gemini', 'XII']
['Gemini', 'XII']
['Gemini', 'XII']
['Apollo', '9']
['Apollo', '9']
['Apollo', '11']
['Apollo', '11']
['Apollo', '12']
['Apollo', '12']
['Apollo', '12']
['Apollo', '14']
['Apollo', '14']
['Apollo', '14']
['Apollo', '15']
['Apollo', '15']
['Apollo', '15']
['Apollo', '15']
['Apollo', '15']
['Apollo', '15']
['Apollo', '16']
['Apollo', '16']
['Apollo', '16']
['Apollo', '16']
['Apollo', '16']
['Apollo', '17']
['Apollo', '17']
['Apollo', '17']
['Apollo', '17']
['Apollo', '17']
['Skylab', '2']
['Skylab', '2']
['Skylab', '2']
['Skylab', '2']
['Skylab', '3']
['Skylab', '3']
['Skylab', '3']
['Skylab', '4']
['Skylab', '4']
['Skylab', '4']
['Skylab', '4']
['STS', '5']
['STS', '6']
['STS', '41B']
['STS', '41B']
['STS', '41C']
['STS', '41C']
['STS', '41G']
['STS', '51A']
['STS', '51A']
['STS', '51D']
['STS', '51I']
['STS', '51I']
['STS', '61B']
['STS', '61B']


In [24]:
spacewalks['Vehicle']= vehicles_cleaned
spacewalks['was_moonshot'] = was_moonshot
spacewalks['crew_0']=crew_0
spacewalks['crew_1']=crew_1
spacewalks['crew_2']=crew_2
spacewalks['spacewalk_duration_in_seconds'] = spacewalk_durationS_in_sec
spacewalks['EVA_start_year'] = EVA_start_year
spacewalks['EVA_start_month'] = EVA_start_month
spacewalks['EVA_start_day'] = EVA_start_day
spacewalks['mission_type'] = mission_type

In [25]:
finaldata = pd.merge(spacewalks, orbitalmissiondata, how='outer', on = ['Vehicle'])

In [26]:
finaldata.to_csv('spacewalk_with_orbital_data.csv', encoding='utf-8')

Proceed to 3_spacewalk_analysis